In [1]:
import sys
sys.path.append("/home/slazien/projects/data_science/fitbit_analysis/")

In [4]:
import os

from src.extractor import unzip_myfitbitdata
from src.constants import PATH_DATA

In [7]:
# Unzip 
unzip_myfitbitdata(os.path.join(PATH_DATA, "MyFitbitData.zip"), os.path.join(PATH_DATA, "fitbit_data_unzipped"))

FileNotFoundError: [Errno 2] No such file or directory: '~/projects/data_science/fitbit_analysis/data/MyFitbitData.zip'